In [1]:
import copy
import itertools as its
import math
import os
import pathlib
import re
import sys
from typing import Dict, List, Optional, Tuple, Union
from collections import Counter, defaultdict, deque

import networkx as nx
import numpy as np
import pandas as pd
from IPython.display import clear_output
from matplotlib import pyplot as plt

from aoc import sim_new as sim, testing, util

twopi = 2 * math.pi

%matplotlib inline

INPUT_PATH = pathlib.Path('..') / 'input' / 'dec22.txt'

In [2]:
# First we do by hand, applying to whole deck

DEAL_NEW = 0
CUT = 1
DEAL_INC = 2

FIND_INT_RE = re.compile(r'-?\d+$')

def read_instruction(line: str) -> Tuple[int, Optional[int]]:
    if line == 'deal into new stack':
        return DEAL_NEW, None
    elif line.startswith('cut'):
        return CUT, int(FIND_INT_RE.findall(line)[0])
    elif line.startswith('deal with increment'):
        return DEAL_INC, int(FIND_INT_RE.findall(line)[0])
    else:
        raise ValueError('Bad instruction')

def read_instructions(text: str) -> List[Tuple[int, Optional[int]]]:
    return list(map(read_instruction, text.strip().split('\n')))

In [3]:
def apply_instruction(stack: List[int], instruction: int, arg: Optional[int]) -> List[int]:
    if instruction == DEAL_NEW:
        return list(reversed(stack))
    elif instruction == CUT:
        return stack[arg:] + stack[:arg]
    elif instruction == DEAL_INC:
        out = [0] * len(stack)
        out_idx = 0
        in_idx = 0
        while in_idx < len(stack):
            out[out_idx] = stack[in_idx]
            out_idx = (out_idx + arg) % len(stack)
            in_idx += 1
        return out
    else:
        raise ValueError('Bad instruction')

def apply_instructions(stack: List[int], instructions: List[Tuple[int, Optional[int]]]):
    for instruction, arg in instructions:
        stack = apply_instruction(stack, instruction, arg)
    return stack

In [4]:
instructions = read_instructions(INPUT_PATH.read_text().strip())
print(f'The answer to part 1 is {apply_instructions(list(range(10007)), instructions).index(2019)}')

The answer to part 1 is 1538


In [5]:
# To do large numbers, we'll simply follow a single card around

def follow_card(card_name: int, deck_size: int, instructions: List[Tuple[int, Optional[int]]], track_seen: bool = False):
    position = card_name
    seen_slots = {position: 0}
    for round_num, (instruction, arg) in enumerate(instructions, 1):
        if instruction == DEAL_NEW:
            position = (-1 - position)
        elif instruction == CUT:
            if arg < 0:
                # Reverse the arg
                arg = deck_size + arg
            position = (position - arg)
        elif instruction == DEAL_INC:
            position = (position * arg)
        else:
            raise ValueError('Bad instruction')
        if track_seen and position in seen_slots:
            return position, seen_slots[position], round_num
        seen_slots[position] = round_num
    return position

In [6]:
assert follow_card(2019, 10007, instructions) % 10007 == 1538

In [7]:
# Note that this is just a linear function % the deck size, so to do
# much larger numbers, we need to figure out what the composed
# linear function is mod the deck size.
#
# But if f(x) = cx + d, then f^n(x) = c^n x + (c^n - 1) / (c - 1) * d
# So if we're careful about % deck_size, then this should be pretty easy

In [8]:
# Modular mutliplicative inverses

def egcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = egcd(b % a, a)
        return (g, x - (b // a) * y, y)

def modinv(a, m):
    g, x, y = egcd(a, m)
    if g != 1:
        raise Exception('modular inverse does not exist')
    else:
        return x % m

In [9]:
# Here we have a function that computes this linear polynomial
def follow_card_many_times(card_name, deck_size, num_rounds):
    c0 = follow_card(0, deck_size, instructions) % deck_size
    c1 = (follow_card(1, deck_size, instructions) - c0) % deck_size
    final_c1 = pow(c1, num_rounds, deck_size)
    final_c0 = ((pow(c1, num_rounds, deck_size) - 1) * modinv(c1 - 1, deck_size) * c0) % deck_size
    return (final_c1 * card_name + final_c0) % deck_size

In [10]:
output = apply_instructions(list(range(10007)), instructions)
assert follow_card_many_times(2019, 10007, 1) == output.index(2019)
output = apply_instructions(output, instructions)
assert follow_card_many_times(2019, 10007, 2) == output.index(2019)

In [11]:
# But the prompt is tricky. It actually wants us to _invert_ this polynomial *waaahhhhh*

In [12]:
def inv_fcmt(card_name, deck_size, num_rounds):
    c0 = follow_card(0, deck_size, instructions) % deck_size
    c1 = (follow_card(1, deck_size, instructions) - c0) % deck_size

    # Invert the polynomial
    c1 = modinv(c1, deck_size)
    c0 = (-c0 * c1) % deck_size

    final_c1 = pow(c1, num_rounds, deck_size)
    final_c0 = ((pow(c1, num_rounds, deck_size) - 1) * modinv(c1 - 1, deck_size) * c0) % deck_size
    return (final_c1 * card_name + final_c0) % deck_size

In [13]:
# Make sure this is actually the inverse function
assert follow_card_many_times(inv_fcmt(2019, 10007, 4), 10007, 4) == 2019

In [14]:
print(f'The answer to part 2 is {inv_fcmt(2020, 119315717514047, 101741582076661)}')

The answer to part 2 is 96196710942473
